# (4주차) 9월30일
> 파이토치를 이용하여 회귀모형 학습하기 (2) 

- toc:true
- branch: master
- badges: true
- comments: true
- author: 최규빈

### 강의영상 

> youtube: https://youtube.com/playlist?list=PLQqh36zP38-ytquucwsAcrSgaPDJedrvF

`-` (1/4) Step 1~2 요약 (1) 

`-` (2/4) Step 1~2 요약 (2), Step 3: derivation  

`-` (3/4) Step 4: update (1) 

`-` (4/4) Step 4: update (2), Step 1~4 를 for 문으로 처리

In [1]:
import torch 
import numpy as np 

### Data

`-` model: $y_i= w_0+w_1 x_i +\epsilon_i = 2.5 + 4x_i +\epsilon_i, \quad i=1,2,\dots,n$ 

`-` model: ${\bf y}={\bf X}{\bf W} +\boldsymbol{\epsilon}$

- ${\bf y}=\begin{bmatrix} y_1 \\ y_2 \\ \dots \\ y_n\end{bmatrix}, \quad {\bf X}=\begin{bmatrix} 1 & x_1 \\ 1 & x_2 \\ \dots \\ 1 & x_n\end{bmatrix}, \quad {\bf W}=\begin{bmatrix} 2.5 \\ 4 \end{bmatrix}, \quad \boldsymbol{\epsilon}= \begin{bmatrix} \epsilon_1 \\ \dots \\ \epsilon_n\end{bmatrix}$

In [2]:
torch.manual_seed(43052)
n=100
ones= torch.ones(n)
x,_ = torch.randn(n).sort()
X = torch.vstack([ones,x]).T
W = torch.tensor([2.5,4])
ϵ = torch.randn(n)*0.5
y = X@W + ϵ
ytrue = X@W

### step1~2 요약 

#### 방법1: 모델을 직접선언 + loss함수도 직접선언 

In [3]:
What1=torch.tensor([-5.0,10.0],requires_grad=True) 
yhat1=X@What1
loss1=torch.mean((y-yhat1)**2) 
loss1

tensor(85.8769, grad_fn=<MeanBackward0>)

#### 방법2: 모델식을 torch.nn으로 선언 (bias=False) + loss 직접선언 

In [4]:
net2=torch.nn.Linear(in_features=2,out_features=1,bias=False) 
net2.weight.data= torch.tensor([[-5.0,10.0]]) 
yhat2=net2(X) 
loss2=torch.mean((y.reshape(100,1)-yhat2)**2) 
loss2

tensor(85.8769, grad_fn=<MeanBackward0>)

#### 방법3: 모델식을 torch.nn으로 선언 (bias=True) + loss 직접선언

In [5]:
net3=torch.nn.Linear(in_features=1,out_features=1,bias=True) 
net3.weight.data= torch.tensor([[10.0]])
net3.bias.data= torch.tensor([[-5.0]]) 
yhat3=net3(x.reshape(100,1)) 
loss3=torch.mean((y.reshape(100,1)-yhat3)**2) 
loss3

tensor(85.8769, grad_fn=<MeanBackward0>)

#### 방법4: 모델식을 직접선언 + loss함수는 torch.nn.MSELoss() 

In [6]:
What4=torch.tensor([-5.0,10.0],requires_grad=True) 
yhat4=X@What4 
lossfn=torch.nn.MSELoss() 
loss4=lossfn(y,yhat4) 
loss4

tensor(85.8769, grad_fn=<MseLossBackward>)

#### 방법5: 모델식을 torch.nn으로 선언 (bias=False) + loss함수는 torch.nn.MSELoss()

In [7]:
net5=torch.nn.Linear(in_features=2,out_features=1,bias=False) 
net5.weight.data= torch.tensor([[-5.0,10.0]]) 
yhat5=net5(X) 
#lossfn=torch.nn.MSELoss() 
loss5=lossfn(y.reshape(100,1),yhat5) 
loss5 

tensor(85.8769, grad_fn=<MseLossBackward>)

#### 방법6: 모델식을 torch.nn으로 선언 (bias=True) + loss함수는 torch.nn.MSELoss()

In [8]:
net6=torch.nn.Linear(in_features=1,out_features=1,bias=True) 
net6.weight.data= torch.tensor([[10.0]])
net6.bias.data= torch.tensor([[-5.0]]) 
yhat6=net6(x.reshape(100,1)) 
loss6=lossfn(y.reshape(100,1),yhat6) 
loss6

tensor(85.8769, grad_fn=<MseLossBackward>)

### step3: derivation - 나중에는 손으로 미분하는 것은 거의 불가능하므로 모든 loss에 대해서 backward() 함수를 이용함.

#### loss1 

In [9]:
loss1.backward()

In [10]:
What1.grad.data

tensor([-13.4225,  11.8893])

- 이것이 손계산을 통한 이론적인 미분값과 일치함은 이전시간에 확인하였음. 

#### loss2

In [11]:
loss2.backward()

In [12]:
net2.weight.grad

tensor([[-13.4225,  11.8893]])

#### loss3 

In [13]:
loss3.backward()

In [14]:
net3.bias.grad,net3.weight.grad

(tensor([[-13.4225]]), tensor([[11.8893]]))

#### loss4

In [15]:
loss4.backward()

In [16]:
What4.grad.data

tensor([-13.4225,  11.8893])

#### loss5 

In [17]:
loss5.backward()

In [18]:
net5.weight.grad

tensor([[-13.4225,  11.8893]])

#### loss6

In [19]:
loss6.backward()

In [20]:
net6.bias.grad,net6.weight.grad

(tensor([[-13.4225]]), tensor([[11.8893]]))

### step4: update 

#### loss1 

In [21]:
What1.data ## update 전 

tensor([-5., 10.])

In [22]:
lr=0.1 
What1.data = What1.data - lr*What1.grad.data ## update 후 
What1

tensor([-3.6577,  8.8111], requires_grad=True)

#### loss2 

In [23]:
net2.weight.data 

tensor([[-5., 10.]])

In [24]:
optmz2 = torch.optim.SGD(net2.parameters(),lr=0.1) 

In [25]:
optmz2.step() ## update 

In [26]:
net2.weight.data ## update 후

tensor([[-3.6577,  8.8111]])

In [45]:
# list(net2.parameters()) 는 
# net2.weight 와 같은 값을 출력한다.

#### loss3

In [27]:
net3.bias.data,net3.weight.data

(tensor([[-5.]]), tensor([[10.]]))

In [28]:
optmz3 = torch.optim.SGD(net3.parameters(),lr=0.1) 

In [29]:
optmz3.step()

In [30]:
net3.bias.data,net3.weight.data

(tensor([[-3.6577]]), tensor([[8.8111]]))

In [31]:
list(net3.parameters()) # 두번째가 bias, 첫번째가 weight로 값이 들어가 있다.

[Parameter containing:
 tensor([[8.8111]], requires_grad=True), Parameter containing:
 tensor([[-3.6577]], requires_grad=True)]

#### loss4

In [32]:
What4.data ## update 전 

tensor([-5., 10.])

In [33]:
lr=0.1 
What4.data = What4.data - lr*What4.grad.data ## update 후 
What4

tensor([-3.6577,  8.8111], requires_grad=True)

#### loss5

In [34]:
net5.weight.data 

tensor([[-5., 10.]])

In [35]:
optmz5 = torch.optim.SGD(net5.parameters(),lr=0.1) 

In [36]:
optmz5.step() ## update 

In [37]:
net5.weight.data ## update 후

tensor([[-3.6577,  8.8111]])

#### loss6

In [38]:
net6.bias.data,net6.weight.data

(tensor([[-5.]]), tensor([[10.]]))

In [39]:
optmz6 = torch.optim.SGD(net6.parameters(),lr=0.1) 

In [40]:
optmz6.step()

In [41]:
net6.bias.data,net6.weight.data

(tensor([[-3.6577]]), tensor([[8.8111]]))

### step1~4를 반복하면된다. 

In [49]:
net=torch.nn.Linear(in_features=2,out_features=1,bias=False) ## 모형정의 
optmz=torch.optim.SGD(net.parameters(),lr=0.1)
mseloss=torch.nn.MSELoss() 
for epoc in range(100): 
    # step1: yhat 
    yhat=net(X) ## yhat 계산 
    # step2: loss
    loss=mseloss(y.reshape(100,1),yhat) 
    # step3: derivation 
    loss.backward() 
    # step4: update
    optmz.step()
    optmz.zero_grad() ## 직전 계산 결과가 계속해서 누적되는 현상을 해결하는 코드 

In [ ]:
list(net.parameters())

[Parameter containing:
 tensor([[2.4459, 4.0043]], requires_grad=True)]

### 숙제 

아래를 실행해보고 결과를 관찰하라. 
```python
net=torch.nn.Linear(in_features=2,out_features=1,bias=False) ## 모형정의 
optmz=torch.optim.SGD(net.parameters(),lr=0.1)
mseloss=torch.nn.MSELoss() 
for epoc in range(100): 
    # step1: yhat 
    yhat=net(X) ## yhat 계산 
    # step2: loss
    loss=mseloss(y.reshape(100,1),yhat) 
    # step3: derivation 
    loss.backward() 
    # step4: update
    optmz.step()
```